In [1]:
import weakref


class ExpensiveObject:

    def __del__(self):
        print('(Deleting {})'.format(self))


obj = ExpensiveObject()
r = weakref.ref(obj)

print('obj:', obj)
print('ref:', r)
print('r():', r())

print('deleting obj')
del obj
print('r():', r())

obj: <__main__.ExpensiveObject object at 0x000001E9062C8F70>
ref: <weakref at 0x000001E9062D75E0; to 'ExpensiveObject' at 0x000001E9062C8F70>
r(): <__main__.ExpensiveObject object at 0x000001E9062C8F70>
deleting obj
(Deleting <__main__.ExpensiveObject object at 0x000001E9062C8F70>)
r(): None


In [2]:
import weakref


class ExpensiveObject:

    def __del__(self):
        print('(Deleting {})'.format(self))


def callback(reference):
    """Invoked when referenced object is deleted"""
    print('callback({!r})'.format(reference))


obj = ExpensiveObject()
r = weakref.ref(obj, callback)

print('obj:', obj)
print('ref:', r)
print('r():', r())

print('deleting obj')
del obj
print('r():', r())

obj: <__main__.ExpensiveObject object at 0x000001E9062DC1F0>
ref: <weakref at 0x000001E9062DE0E0; to 'ExpensiveObject' at 0x000001E9062DC1F0>
r(): <__main__.ExpensiveObject object at 0x000001E9062DC1F0>
deleting obj
(Deleting <__main__.ExpensiveObject object at 0x000001E9062DC1F0>)
callback(<weakref at 0x000001E9062DE0E0; dead>)
r(): None


In [ ]:
import os
os._exit(00)

In [7]:
import sys
import weakref


class ExpensiveObject:

    def __del__(self):
        print('(Deleting {})'.format(self))


def on_finalize(*args):
    print('on_finalize({!r})'.format(args))


obj = ExpensiveObject()
f = weakref.finalize(obj, on_finalize, 'extra argument')
f.atexit = True

(Deleting <__main__.ExpensiveObject object at 0x00000252896ABA00>)
on_finalize(('extra argument',))


In [ ]:
import os
os._exit(00)

In [1]:
import sys
import weakref


class ExpensiveObject:

    def __del__(self):
        print('(Deleting {})'.format(self))


def on_finalize(*args):
    print('on_finalize({!r})'.format(args))

obj = ExpensiveObject()
f = weakref.finalize(obj, on_finalize, 'extra argument')
f.atexit = False

In [ ]:
import os
os._exit(00)

In [2]:
import gc
import weakref


class ExpensiveObject:

    def __del__(self):
        print('(Deleting {})'.format(self))


def on_finalize(*args):
    print('on_finalize({!r})'.format(args))


obj = ExpensiveObject()
obj_id = id(obj)
f = weakref.finalize(obj, on_finalize, obj)
f.atexit = False

del obj

for o in gc.get_objects():
    if id(o) == obj_id:
        print('found uncollected object in gc')

found uncollected object in gc


In [ ]:
import os
os._exit(00)

In [2]:
import gc
import weakref


class ExpensiveObject:

    def __del__(self):
        print('(Deleting {})'.format(self))

    def do_finalize(self):
        print('do_finalize')


obj = ExpensiveObject()
obj_id = id(obj)

f = weakref.finalize(obj, obj.do_finalize)
f.atexit = False

del obj

for o in gc.get_objects():
    if id(o) == obj_id:
        print('found uncollected object in gc')

found uncollected object in gc


In [3]:
import weakref


class ExpensiveObject:

    def __init__(self, name):
        self.name = name

    def __del__(self):
        print('(Deleting {})'.format(self))


obj = ExpensiveObject('My Object')
r = weakref.ref(obj)
p = weakref.proxy(obj)

print('via obj:', obj.name)
print('via ref:', r().name)
print('via proxy:', p.name)
del obj
print('via proxy:', p.name)

via obj: My Object
via ref: My Object
via proxy: My Object
(Deleting <__main__.ExpensiveObject object at 0x000001B42DCFBD00>)


ReferenceError: weakly-referenced object no longer exists

In [ ]:
import os
os._exit(00)

In [1]:
import gc
from pprint import pprint
import weakref

gc.set_debug(gc.DEBUG_UNCOLLECTABLE)


class ExpensiveObject:

    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return 'ExpensiveObject({})'.format(self.name)

    def __del__(self):
        print('    (Deleting {})'.format(self))


def demo(cache_factory):
    # hold objects so any weak references
    # are not removed immediately
    all_refs = {}
    # create the cache using the factory
    print('CACHE TYPE:', cache_factory)
    cache = cache_factory()
    for name in ['one', 'two', 'three']:
        o = ExpensiveObject(name)
        cache[name] = o
        all_refs[name] = o
        del o  # decref

    print('  all_refs =', end=' ')
    pprint(all_refs)
    print('\n  Before, cache contains:', list(cache.keys()))
    for name, value in cache.items():
        print('    {} = {}'.format(name, value))
        del value  # decref

    # remove all references to the objects except the cache
    print('\n  Cleanup:')
    del all_refs
    gc.collect()

    print('\n  After, cache contains:', list(cache.keys()))
    for name, value in cache.items():
        print('    {} = {}'.format(name, value))
    print('  demo returning')
    return


demo(dict)
print()

demo(weakref.WeakValueDictionary)

CACHE TYPE: <class 'dict'>
  all_refs = {'one': ExpensiveObject(one),
 'three': ExpensiveObject(three),
 'two': ExpensiveObject(two)}

  Before, cache contains: ['one', 'two', 'three']
    one = ExpensiveObject(one)
    two = ExpensiveObject(two)
    three = ExpensiveObject(three)

  Cleanup:

  After, cache contains: ['one', 'two', 'three']
    one = ExpensiveObject(one)
    two = ExpensiveObject(two)
    three = ExpensiveObject(three)
  demo returning
    (Deleting ExpensiveObject(one))
    (Deleting ExpensiveObject(two))
    (Deleting ExpensiveObject(three))

CACHE TYPE: <class 'weakref.WeakValueDictionary'>
  all_refs = {'one': ExpensiveObject(one),
 'three': ExpensiveObject(three),
 'two': ExpensiveObject(two)}

  Before, cache contains: ['one', 'two', 'three']
    one = ExpensiveObject(one)
    two = ExpensiveObject(two)
    three = ExpensiveObject(three)

  Cleanup:
    (Deleting ExpensiveObject(one))
    (Deleting ExpensiveObject(two))
    (Deleting ExpensiveObject(three))

  A

In [5]:
import gc
from pprint import pprint
import weakref

class Graph:
    def __init__(self, name):
        self.name = name
        self.other = None
    def set_next(self, other):
        print(f"{self.name}.set_next({self.other}, {type(other)})")
        self.other = other
    def all_nodes(self):
        "Generate the nodes in the graph sequence."
        yield self
        n = self.other
        while n and n.name != self.name:
            yield n
            n = n.other
        if n is self:
            yield n
        return
    def __str__(self):
        return "->".join([n.name for n in self.all_nodes()])
    def __repr__(self):
        return f"{self.__class__.__name__}({self.name})"
    def __del__(self):
        print (f'(Deleting {self.name})')
        self.set_next(None)
        
        
class WeakGraph(Graph):
    def set_next(self, other):
        if other is not None:
            if self in other.all_nodes():
                other = weakref.proxy(other)
        super(WeakGraph, self).set_next(other)
        return
    
def collect_and_show_garbage():
    "show what garbage is present."
    print("Collecting...")
    n = gc.collect()
    print("Unreachable objects:", n)
    print("Garbage:")
    pprint(gc.garbage)

def demo(graph_factory):
    print("Set up graph:")
    one = graph_factory("one")
    two = graph_factory("two")
    three = graph_factory("three")
    one.set_next(two)
    two.set_next(three)
    three.set_next(one)
    
    print("\nGraphs:")
    print(str(one))
    print(str(two))
    print(str(three))
    collect_and_show_garbage()     
    
    print('\n')
    three = None
    two = None
    print("After 2 references removed:")
    print(str(one))
    collect_and_show_garbage()     
    
    print("\n")
    print("Removing last reference:")
    one = None
    collect_and_show_garbage()      

gc: collectable <tuple 0x00000251DE317460>
gc: collectable <ValueError 0x00000251DE3C5360>
gc: collectable <traceback 0x00000251DE265A80>
gc: collectable <frame 0x00000251DE3F2840>
gc: collectable <traceback 0x00000251DE2CAB80>
gc: collectable <tuple 0x00000251DE37D790>
gc: collectable <ValueError 0x00000251DE367400>
gc: collectable <traceback 0x00000251DE2CA100>
gc: collectable <frame 0x00000251DCA0D530>
gc: collectable <traceback 0x00000251DE2CAFC0>
gc: collectable <frame 0x00000251DE3F2C40>
gc: collectable <traceback 0x00000251DE2CA7C0>
gc: collectable <frame 0x00000251DE3F6040>
gc: collectable <frame 0x00000251DDBBB470>
gc: collectable <frame 0x00000251DDBD2350>


In [6]:
gc.set_debug(gc.DEBUG_LEAK)

In [7]:
print ('Setting up the cycle\n')
demo(Graph)
print('\nBreaking the cycle and cleaning up garbage\n')
gc.garbage[0].set_next(None)
while gc.garbage:
    del gc.garbage[0]
print('\n')
collect_and_show_garbage()

Setting up the cycle

Set up graph:
one.set_next(None, <class '__main__.Graph'>)
two.set_next(None, <class '__main__.Graph'>)
three.set_next(None, <class '__main__.Graph'>)

Graphs:
one->two->three->one
two->three->one->two
three->one->two->three
Collecting...
Unreachable objects: 58
Garbage:
[('<built-in function default_int_handler> is not a valid Handlers',),
 ValueError('<built-in function default_int_handler> is not a valid Handlers'),
 <frame at 0x00000251DE27CA60, file 'D:\\Anaconda\\lib\\enum.py', line 608, code _missing_>,
 ('<built-in function default_int_handler> is not a valid Handlers',),
 ValueError('<built-in function default_int_handler> is not a valid Handlers'),
 <frame at 0x00000251DCA0D740, file 'D:\\Anaconda\\lib\\enum.py', line 595, code __new__>,
 <frame at 0x00000251DE275C40, file 'D:\\Anaconda\\lib\\enum.py', line 304, code __call__>,
 <frame at 0x00000251DE24B780, file 'D:\\Anaconda\\lib\\signal.py', line 32, code _int_to_enum>,
 <frame at 0x00000251DE279C80, 

gc: collectable <type 0x00000251DD9A06D0>
gc: collectable <dict 0x00000251DE359EC0>
gc: collectable <tuple 0x00000251DE34F400>
gc: collectable <tuple 0x00000251DE32B580>
gc: collectable <function 0x00000251DE32DDC0>
gc: collectable <tuple 0x00000251DE2FE850>
gc: collectable <cell 0x00000251DE356B20>
gc: collectable <type 0x00000251DD9A0A80>
gc: collectable <dict 0x00000251DE359FC0>
gc: collectable <tuple 0x00000251DE246480>
gc: collectable <function 0x00000251DE32DA60>
gc: collectable <function 0x00000251DE32DAF0>
gc: collectable <function 0x00000251DE32DB80>
gc: collectable <function 0x00000251DE32DC10>
gc: collectable <function 0x00000251DE32DCA0>
gc: collectable <function 0x00000251DE32DD30>
gc: collectable <getset_descriptor 0x00000251DE35C6C0>
gc: collectable <getset_descriptor 0x00000251DE35C7C0>
gc: collectable <tuple 0x00000251DE2FECA0>
gc: collectable <ValueError 0x00000251DE3752C0>
gc: collectable <traceback 0x00000251DE31EA40>
gc: collectable <frame 0x00000251DE328A40>
gc: c

(Deleting one)
one.set_next(two->three->one->two, <class 'NoneType'>)
(Deleting two)
two.set_next(three->one, <class 'NoneType'>)
(Deleting three)
three.set_next(one, <class 'NoneType'>)
Unreachable objects: 0
Garbage:
[('<built-in function default_int_handler> is not a valid Handlers',),
 ValueError('<built-in function default_int_handler> is not a valid Handlers'),
 <frame at 0x00000251DE27CA60, file 'D:\\Anaconda\\lib\\enum.py', line 608, code _missing_>,
 ('<built-in function default_int_handler> is not a valid Handlers',),
 ValueError('<built-in function default_int_handler> is not a valid Handlers'),
 <frame at 0x00000251DCA0D740, file 'D:\\Anaconda\\lib\\enum.py', line 595, code __new__>,
 <frame at 0x00000251DE275C40, file 'D:\\Anaconda\\lib\\enum.py', line 304, code __call__>,
 <frame at 0x00000251DE24B780, file 'D:\\Anaconda\\lib\\signal.py', line 32, code _int_to_enum>,
 <frame at 0x00000251DE279C80, file 'D:\\Anaconda\\lib\\signal.py', line 48, code signal>,
 <frame at 0x00

gc: collectable <dict 0x00000251DE407880>
gc: collectable <Graph 0x00000251DE2FEE20>
gc: collectable <dict 0x00000251DE407940>
gc: collectable <Graph 0x00000251DE2FEBB0>
gc: collectable <dict 0x00000251DE4079C0>


AttributeError: 'tuple' object has no attribute 'set_next'

In [8]:
demo(WeakGraph)

Set up graph:
one.set_next(None, <class '__main__.WeakGraph'>)
two.set_next(None, <class '__main__.WeakGraph'>)
three.set_next(None, <class 'weakproxy'>)

Graphs:
one->two->three
two->three->one->two
three->one->two->three
Collecting...
Unreachable objects: 28
Garbage:
[('<built-in function default_int_handler> is not a valid Handlers',),
 ValueError('<built-in function default_int_handler> is not a valid Handlers'),
 <frame at 0x00000251DE27CA60, file 'D:\\Anaconda\\lib\\enum.py', line 608, code _missing_>,
 ('<built-in function default_int_handler> is not a valid Handlers',),
 ValueError('<built-in function default_int_handler> is not a valid Handlers'),
 <frame at 0x00000251DCA0D740, file 'D:\\Anaconda\\lib\\enum.py', line 595, code __new__>,
 <frame at 0x00000251DE275C40, file 'D:\\Anaconda\\lib\\enum.py', line 304, code __call__>,
 <frame at 0x00000251DE24B780, file 'D:\\Anaconda\\lib\\signal.py', line 32, code _int_to_enum>,
 <frame at 0x00000251DE279C80, file 'D:\\Anaconda\\lib\

gc: collectable <traceback 0x00000251DE352AC0>
gc: collectable <AttributeError 0x00000251DE2B89F0>
gc: collectable <traceback 0x00000251DE352B00>
gc: collectable <frame 0x00000251DC8C8250>
gc: collectable <frame 0x00000251DC952A00>
gc: collectable <frame 0x00000251DDD90420>
gc: collectable <list 0x00000251DE2BC0C0>
gc: collectable <dict 0x00000251DE41A340>
gc: collectable <dict 0x00000251DE41A140>
gc: collectable <list 0x00000251DE41D600>
gc: collectable <ExecutionResult 0x00000251DE2FED00>
gc: collectable <dict 0x00000251DCEAB2C0>
gc: collectable <ExecutionInfo 0x00000251DE2FEC70>
gc: collectable <tuple 0x00000251DE37D880>
gc: collectable <ValueError 0x00000251DE35B310>
gc: collectable <traceback 0x00000251DE407500>
gc: collectable <frame 0x00000251DDD8FB60>
gc: collectable <traceback 0x00000251DE2BF2C0>
gc: collectable <tuple 0x00000251DE32BC70>
gc: collectable <ValueError 0x00000251DE35B360>
gc: collectable <traceback 0x00000251DE2BF240>
gc: collectable <frame 0x00000251DBD2BF40>
gc

,
              'user_expressions': {}},
  'header': {'date': datetime.datetime(2021, 2, 17, 1, 22, 9, 691272, tzinfo=datetime.timezone.utc),
             'msg_id': 'bcfba0d3a446416080b1d86dc96204f8',
             'msg_type': 'execute_request',
             'session': '59a877b36e7a49ecaf785dfc409ef2c5',
             'username': 'username',
             'version': '5.2'},
  'metadata': {},
  'msg_id': 'bcfba0d3a446416080b1d86dc96204f8',
  'msg_type': 'execute_request',
  'parent_header': {}},
 {'allow_stdin': True,
  'code': "print ('Setting up the cycle\\n')\n"
          'demo(Graph)\n'
          "print('\\nBreaking the cycle and cleaning up garbage\\n')\n"
          'gc.garbage[0].set_next(None)\n'
          'while gc.garbage:\n'
          '    del gc.garbage[0]\n'
          "print('\\n')\n"
          'collect_and_show_garbage()',
  'silent': False,
  'stop_on_error': True,
  'store_history': True,
  'user_expressions': {}},
 [],
 <ExecutionResult object at 251de2fed00, execution_coun